# TMDb data prep

This notebook will be used to prepare a dataset to use to train the supervised learning model for our MSCI 446 final project

## first part: just get page 1 of the discover TV shows and match up with details and then save to .csv for further processing

In [2]:
import requests

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
apikey = os.getenv("API_KEY")

In [5]:
ploads = {'api_key':  apikey, 'sort_by': 'popularity.desc'}
r = requests.get('https://api.themoviedb.org/3/discover/tv', params=ploads)
print(r.url)

https://api.themoviedb.org/3/discover/tv?api_key=83930268a198760b8b6ed65003b56157&sort_by=popularity.desc


In [6]:
import json

In [7]:
resp = r.json()

In [8]:
print(resp)

{'page': 1, 'results': [{'backdrop_path': '/9ijMGlJKqcslswWUzTEwScm82Gs.jpg', 'first_air_date': '2019-11-12', 'genre_ids': [10765, 10759, 37], 'id': 82856, 'name': 'The Mandalorian', 'origin_country': ['US'], 'original_language': 'en', 'original_name': 'The Mandalorian', 'overview': 'After the fall of the Galactic Empire, lawlessness has spread throughout the galaxy. A lone gunfighter makes his way through the outer reaches, earning his keep as a bounty hunter.', 'popularity': 2252.745, 'poster_path': '/sWgBv7LV2PRoQgkxwlibdGXKz1S.jpg', 'vote_average': 8.5, 'vote_count': 3525}, {'backdrop_path': '/edmk8xjGBsYVIf4QtLY9WMaMcXZ.jpg', 'first_air_date': '2005-03-27', 'genre_ids': [18], 'id': 1416, 'name': "Grey's Anatomy", 'origin_country': ['US'], 'original_language': 'en', 'original_name': "Grey's Anatomy", 'overview': 'Follows the personal and professional lives of a group of doctors at Seattle’s Grey Sloan Memorial Hospital.', 'popularity': 897.866, 'poster_path': '/clnyhPqj1SNgpAdeSS6a

In [10]:
resp['results'][0]['original_name']

'The Mandalorian'

## resp reference now holds the response object which can be used directly

In [11]:
import pandas as pd

In [12]:
df = pd.DataFrame(resp['results'])

In [13]:
df.head()

,backdrop_path,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,overview,popularity,poster_path,vote_average,vote_count
0,/9ijMGlJKqcslswWUzTEwScm82Gs.jpg,2019-11-12,"[10765, 10759, 37]",82856,The Mandalorian,[US],en,The Mandalorian,"After the fall of the Galactic Empire, lawless...",2252.745,/sWgBv7LV2PRoQgkxwlibdGXKz1S.jpg,8.5,3525
1,/edmk8xjGBsYVIf4QtLY9WMaMcXZ.jpg,2005-03-27,[18],1416,Grey's Anatomy,[US],en,Grey's Anatomy,Follows the personal and professional lives of...,897.866,/clnyhPqj1SNgpAdeSS6a6fwE6Bo.jpg,8.1,4532
2,/iDbIEpCM9nhoayUDTwqFL1iVwzb.jpg,2017-09-25,[18],71712,The Good Doctor,[US],en,The Good Doctor,A young surgeon with Savant syndrome is recrui...,880.496,/6tfT03sGp9k4c0J3dypjrI8TSAI.jpg,8.6,5939
3,/ta5oblpMlEcIPIS2YGcq9XEkWK2.jpg,2016-01-25,"[80, 10765]",63174,Lucifer,[US],en,Lucifer,"Bored and unhappy as the Lord of Hell, Lucifer...",550.640,/4EYPN5mVIhKLfxGruy7Dy41dTVn.jpg,8.5,6628
4,/34OGjFEbHj0E3lE2w0iTUVq0CBz.jpg,2020-10-23,[18],87739,The Queen's Gambit,[US],en,The Queen's Gambit,"In a Kentucky orphanage in the 1950s, a young ...",474.811,/zU0htwkhNvBQdVSIKB9s6hgVeFK.jpg,8.7,770


## Get movie details for every entry in results

In [14]:
ploads = {'api_key': apikey}

right = pd.DataFrame()

In [15]:
details = []
# API call for every ID, store all results in array of objects
for index, row in df.iterrows():
    print(row['id'])
    r = requests.get('https://api.themoviedb.org/3/tv/%s' % row['id'], params=ploads)
    details.append(r.json())
    

82856
1416
71712
63174
87739
89844
62286
85552
68507
91605
75006
76479
69050
456
83851
1399
48866
18165
1622
94305


In [22]:
type(details[0]['genres'])

list

In [28]:
type(details[0]['episode_run_time'])

list

In [29]:
details[0]['episode_run_time']

[35, 48]

In [16]:
detailsDF = pd.DataFrame(details)

In [17]:
columns_to_use = detailsDF.columns.difference(df.columns)

In [18]:
complete_results = pd.merge(df, detailsDF, how='inner', on=['id','backdrop_path','first_air_date','original_language', 'popularity', 'poster_path','overview', 'vote_count','vote_average','original_name', 'name'])

In [19]:
complete_results.head()

,backdrop_path,first_air_date,genre_ids,id,name,origin_country_x,original_language,original_name,overview,popularity,...,number_of_episodes,number_of_seasons,origin_country_y,production_companies,production_countries,seasons,spoken_languages,status,tagline,type
0,/wgEWTYAAY7F04o4F20j0L1DGB8i.jpg,2019-07-23,[18],91605,I Am...,[],en,I Am...,Each hour-long film follows a different women ...,361.296,...,3,1,[],[],[],"[{'air_date': '2019-07-23', 'episode_count': 3...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,,Miniseries
1,/nVRyd8hlg0ZLxBn9RaI7mUMQLnz.jpg,2005-09-13,"[18, 9648, 10765]",1622,Supernatural,[US],en,Supernatural,"When they were boys, Sam and Dean Winchester l...",287.304,...,327,15,[US],"[{'id': 38398, 'logo_path': None, 'name': 'Kri...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'air_date': None, 'episode_count': 4, 'id': ...","[{'english_name': 'English', 'iso_639_1': 'en'...",Ended,Between darkness and deliverance,Scripted


In [40]:
complete_results.columns

Index(['id', 'backdrop_path', 'genre_ids', 'first_air_date',
       'original_language', 'popularity', 'poster_path', 'overview',
       'vote_count', 'vote_average', 'original_name', 'origin_country_x',
       'name', 'created_by', 'episode_run_time', 'genres', 'homepage',
       'in_production', 'languages', 'last_air_date', 'last_episode_to_air',
       'next_episode_to_air', 'networks', 'number_of_episodes',
       'number_of_seasons', 'origin_country_y', 'production_companies',
       'production_countries', 'seasons', 'spoken_languages', 'status',
       'tagline', 'type'],
      dtype='object')

In [23]:
complete_results['genres'].head()

0                        [{'id': 18, 'name': 'Drama'}]
1    [{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...
Name: genres, dtype: object

In [24]:
type(complete_results['genres'])

pandas.core.series.Series

In [27]:
complete_results['genres'][0][0]['id']

18

In [30]:
type(complete_results['episode_run_time'])

pandas.core.series.Series

In [31]:
complete_results['episode_run_time'][0]

[50]

## Repeat this process for all 500 pages in original request

## iterate over the next 2-500 pages, append to complete_results df

In [43]:
import time

In [46]:
frames = []

for i in range(2, 501):
    ploads = {'api_key':  apikey, 'sort_by': 'popularity.desc', 'page':i}
    r = requests.get('https://api.themoviedb.org/3/discover/tv', params=ploads)
    resp = r.json()
    left = pd.DataFrame(resp['results'])
    
    details = []
    # API call for every ID, store all results in array of objects
    for index, row in left.iterrows():
        r = requests.get('https://api.themoviedb.org/3/tv/%s' % row['id'], params=ploads)
        details.append(r.json())
    right = pd.DataFrame(details)
    merged = pd.merge(left, right, how='inner', on=['id','backdrop_path','first_air_date','original_language', 'popularity', 'poster_path','overview', 'vote_count','vote_average','original_name', 'name'])
    frames.append(merged)
    time.sleep(5)
    

In [47]:
result = pd.concat(frames)

In [48]:
result.head()

,id,popularity,vote_count,name,first_air_date,backdrop_path,overview,genre_ids,original_name,original_language,...,number_of_episodes,number_of_seasons,origin_country_y,production_companies,production_countries,seasons,spoken_languages,status,tagline,type
0,76773,238.249,550,Station 19,2018-03-22,/PB1agKtni7zo4fea8yk6y4iE5L.jpg,A group of heroic firefighters at Seattle Fire...,"[10759, 18]",Station 19,en,...,46,4,[US],"[{'id': 34209, 'logo_path': '/chuOOQxixQL4fa9L...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'air_date': '2018-03-22', 'episode_count': 1...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,Courage under fire.,Scripted
1,71790,230.154,303,S.W.A.T.,2017-11-02,/gJSqr2prvTegFbL0cEWK9Qti3Vs.jpg,A locally born and bred S.W.A.T. lieutenant is...,"[10759, 80, 18]",S.W.A.T.,en,...,71,4,[US],"[{'id': 11073, 'logo_path': '/wHs44fktdoj6c378...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'air_date': '2017-11-02', 'episode_count': 2...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,Serve • Protect • Unite,Scripted
2,65494,253.691,647,The Crown,2016-11-04,/4InrdamBEM31unNiuEHGYTPX1e2.jpg,"The gripping, decades-spanning inside story of...",[18],The Crown,en,...,40,4,"[GB, US]","[{'id': 5340, 'logo_path': '/zHQ48Hg1In6SJB7WW...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'air_date': '2016-11-04', 'episode_count': 1...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,Times change. Duty endures.,Scripted
3,60735,231.088,6222,The Flash,2014-10-07,/z59kJfcElR9eHO9rJbWp4qWMuee.jpg,After a particle accelerator causes a freak st...,"[18, 10765]",The Flash,en,...,134,7,[US],"[{'id': 1957, 'logo_path': '/3T19XSr6yqaLNK8uJ...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'air_date': '2016-04-19', 'episode_count': 5...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,The fastest man alive.,Scripted
4,44006,214.268,965,Chicago Fire,2012-10-10,/W5geO8tq68ng4TsXyZ81phcPiz.jpg,An edge-of-your-seat view into the lives of ev...,[18],Chicago Fire,en,...,182,9,[US],"[{'id': 26727, 'logo_path': '/jeTxdjXhzgKZyLr3...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'air_date': '2012-10-10', 'episode_count': 2...","[{'english_name': 'English', 'iso_639_1': 'en'...",Returning Series,Feel the heat,Scripted


In [51]:
result.describe

<bound method NDFrame.describe of        id  popularity  vote_count             name first_air_date  \
0   76773     238.249         550       Station 19     2018-03-22   
1   71790     230.154         303         S.W.A.T.     2017-11-02   
2   65494     253.691         647        The Crown     2016-11-04   
3   60735     231.088        6222        The Flash     2014-10-07   
4   44006     214.268         965     Chicago Fire     2012-10-10   
..    ...         ...         ...              ...            ...   
13  64614      18.248           2  En mode Salvail     2013-10-28   
14  62565      14.382         172  My Love Story!!     2015-04-09   
15   2343      14.652          49            Lewis     2007-02-18   
16  85854      12.630          31        The Widow     2019-02-28   
17   3476      15.592          64  Inspector Morse     1987-01-06   

                       backdrop_path  \
0    /PB1agKtni7zo4fea8yk6y4iE5L.jpg   
1   /gJSqr2prvTegFbL0cEWK9Qti3Vs.jpg   
2   /4InrdamBEM31

In [56]:
result.to_csv(path_or_buf='tmdb_tv_dataset_small.csv', index=False)